# problem


create data set randomly

In [76]:
import numpy as np
import pandas as pd

np.random.seed(42)

num_customers = 20

customer_coords = np.random.rand(num_customers, 2) * 100

depot_coords = np.array([[50, 50]])

all_coords = np.vstack([depot_coords, customer_coords])

coords_df = pd.DataFrame(all_coords, columns=['x', 'y'])

coords_df.to_csv('vrp_data.csv', index=False)

print(coords_df)

            x          y
0   50.000000  50.000000
1   37.454012  95.071431
2   73.199394  59.865848
3   15.601864  15.599452
4    5.808361  86.617615
5   60.111501  70.807258
6    2.058449  96.990985
7   83.244264  21.233911
8   18.182497  18.340451
9   30.424224  52.475643
10  43.194502  29.122914
11  61.185289  13.949386
12  29.214465  36.636184
13  45.606998  78.517596
14  19.967378  51.423444
15  59.241457   4.645041
16  60.754485  17.052412
17   6.505159  94.888554
18  96.563203  80.839735
19  30.461377   9.767211
20  68.423303  44.015249


# encoding

In [77]:
import numpy as np
import pandas as pd

coords_df = pd.read_csv('vrp_data.csv')

all_coords = coords_df.values

customer_indices = np.arange(0, len(all_coords))

customer_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [78]:
import plotly.express as px

fig = px.scatter(coords_df, x='x', y='y', color=['Depot'] + ['Customer']*num_customers, 
                 labels={'color': 'Location Type'}, title='Customer and Depot Locations')

fig.show()


# population intilaztion

In [79]:
def initialize_population(pop_size, num_customers):
    customer_indices = np.arange(1, num_customers + 1)  # Exclude depot
    population = []
    for _ in range(pop_size):
        route = np.random.permutation(customer_indices)  # Random customer order
        route = np.concatenate(([0], route, [0]))       # Add depot at start and end
        population.append(route)

    return np.array(population)


# huristic function

In [80]:
def calculate_distance(route, coords):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += np.linalg.norm(coords[route[i]] - coords[route[i + 1]])
    return total_distance


# selection function

In [81]:
import random

def selection(population, fitness_scores, num_parents):
    sorted_indices = np.argsort(fitness_scores)
    ranked_population = [population[i] for i in sorted_indices]
    selected_parents = ranked_population[:num_parents]
    return selected_parents

# crossover function

In [82]:
def crossover(parent1, parent2):
    size = len(parent1) - 2  

    point = np.random.randint(1, size)  

    child = [-1] * len(parent1) 
    child[0], child[-1] = 0, 0  
    child[1:point + 1] = parent1[1:point + 1]  
    pointer = point + 1
    
    for gene in parent2:
        if gene not in child:
            child[pointer] = gene
            pointer += 1
    return np.array(child)


In [83]:
def mutate(route, mutation_rate):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(range(1, len(route) - 1), 2, replace=False)
        route[idx1], route[idx2] = route[idx2], route[idx1]
    return route


<h1>evlolve</h1>
<p>
    <!-- <img src="spinning-cat.gif" alt="Spinning Cat"> -->
</p>

<style>
    h1 {
        color: red;
    }
    p {
        position: relative;
        /* animation: moveRightLeft 2s infinite ease-in-out; */
    }
    img
    {
        width :100px;
        height :100px;
    }

    @keyframes moveRightLeft {
        0% {
            left: 0;
        }
        50% {
            left: 100px;
        }
        100% {
            left: 0;
        }
    }
</style>

In [84]:
def validate_and_repair(route, num_customers):
    all_customers = set(range(1, num_customers + 1))
    visited = set(route[1:-1])  # Exclude depot
    missing_customers = list(all_customers - visited)
    duplicates = [c for c in route[1:-1] if route[1:-1].tolist().count(c) > 1]

    for i in range(1, len(route) - 1):
        if route[i] in duplicates:
            route[i] = missing_customers.pop()
            duplicates.remove(route[i])
    return route


In [85]:
def evolve_population(population, coords, num_customers, mutation_rate, retain_rate):
    fitness_scores = [1 / calculate_distance(route, coords) for route in population]

    sorted_indices = np.argsort(fitness_scores)[::-1]
    retain_length = int(len(population) * retain_rate)
    parents = [population[i] for i in sorted_indices[:retain_length]]

    children = []
    while len(children) < len(population) - len(parents):

        p1, p2 = parents[0], parents[1]  # Just use the first two parents

        child = crossover(p1, p2)
        children.append(child)


    next_gen = parents + children
    next_gen = [validate_and_repair(mutate(route, mutation_rate), num_customers) for route in next_gen]

    return np.array(next_gen)


# trying the alogrithm 

In [86]:
import numpy as np

def run_ga(coords, num_customers, population_size=100, generations=1000, mutation_rate=0.02, retain_rate=0.2):
    
    population = initialize_population(population_size, num_customers)

    best_route = None
    best_distance = float('inf')
    best_distance_arr = []

    for gen in range(generations):
        population = evolve_population(population, coords, num_customers, mutation_rate, retain_rate)

        distances = [calculate_distance(route, coords) for route in population]
        min_distance = min(distances)

        # Update the best solution
        if min_distance < best_distance:
            best_distance = min_distance
            best_route = population[np.argmin(distances)]

        if gen % 50 == 0 or gen == generations - 1:
            print(f"Generation {gen + 1}/{generations}: Best Distance = {best_distance:.2f}")
            best_distance_arr.append(best_distance)

    best_distance_arr = np.array(best_distance_arr)
    return best_route, best_distance, best_distance_arr

In [87]:



coords = coords_df[['x', 'y']].to_numpy()

num_customers = len(coords) - 1

best_route, best_distance, best_distance_arr = run_ga(coords, num_customers)
print("maybe a best route:", best_route)
print("a maybe  best distance  ??:", best_distance)


Generation 1/1000: Best Distance = 838.06
Generation 51/1000: Best Distance = 585.57
Generation 101/1000: Best Distance = 580.70
Generation 151/1000: Best Distance = 580.70
Generation 201/1000: Best Distance = 568.20
Generation 251/1000: Best Distance = 480.11
Generation 301/1000: Best Distance = 478.34
Generation 351/1000: Best Distance = 478.34
Generation 401/1000: Best Distance = 478.34
Generation 451/1000: Best Distance = 478.34
Generation 501/1000: Best Distance = 478.34
Generation 551/1000: Best Distance = 478.34
Generation 601/1000: Best Distance = 478.34
Generation 651/1000: Best Distance = 478.34
Generation 701/1000: Best Distance = 478.34
Generation 751/1000: Best Distance = 478.34
Generation 801/1000: Best Distance = 478.34
Generation 851/1000: Best Distance = 478.34
Generation 901/1000: Best Distance = 478.34
Generation 951/1000: Best Distance = 478.34
Generation 1000/1000: Best Distance = 478.34
maybe a best route: [ 0  4  6 17  1 13  5 18  2 20  7 19  3  8 12 14  9 10 15 

<p>
    <img src="spinning-cat.gif" alt="Spinning Cat"> 
</p>
<h1>Loading...</h1>

<style>
    h1 {
        color: red;
        position: relative;
        animation: moveRightLeft 2s infinite ease-in-out; 
    }
    p {

    }
    img {
        width: 100px;
        height: 100px;
    }

    @keyframes moveRightLeft {
        0% {
            left: 0;
        }
        50% {
            left: 100px;
        }
        100% {
            left: 0;
        }
    }
</style>

In [88]:
import plotly.express as px

loss_df = pd.DataFrame({'Generation': np.arange(0, len(best_distance_arr) * 50, 50), 'Best Distance': best_distance_arr})

plot = px.line(loss_df, x='Generation', y='Best Distance', title='Best Distance Over Generations')
plot.show()


In [89]:
import plotly.graph_objects as go

def plot_route(coords_df, best_route):


    route_coords = coords_df.to_numpy()
    route_x = [route_coords[i][0] for i in best_route]
    route_y = [route_coords[i][1] for i in best_route]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=route_x,
        y=route_y,
        mode='markers+lines',
        name='Route',
        marker=dict(size=10, color='blue'),
        line=dict(color='orange', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=[route_x[0], route_x[-1]],  
        y=[route_y[0], route_y[-1]],  
        mode='markers',
        name='Depot',
        marker=dict(size=12, color='red')
    ))

    fig.update_layout(
        title='Best Route Plot',
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        showlegend=True
    )

    # Show plot
    fig.show()


In [90]:
plot_route(coords_df, best_route)


# differntial evolution

In [91]:
#Practical Advice: (e.g., start with NP = 10 * D, and CR = 0.9, F = 0.8)

num_customers = len(coords) - 1
num_populations = 10 * num_customers
num_populations



200

intialze population for differntial evolution

In [92]:
diff_population=initialize_population(num_populations, num_customers)
diff_population

array([[ 0,  9,  8, ..., 19, 11,  0],
       [ 0, 20, 18, ..., 14, 11,  0],
       [ 0, 17, 15, ..., 20,  5,  0],
       ...,
       [ 0, 15,  8, ..., 19, 12,  0],
       [ 0,  9, 17, ..., 10, 15,  0],
       [ 0,  7, 10, ..., 18,  5,  0]])

In [93]:
def run_de(coords, num_customers=20, population_size=200, generations=1000, mutate=0.5, recombination=0.7):
    population = initialize_population(population_size, num_customers)
    best_distance_arr = []
    for gen in range(generations):

        mutate = 0.5 + (0.5 * (gen / generations))
        recombination = 0.9 - (0.4 * (gen / generations))

        
        gen_scores = [] # score keeping
        # cycle through each individual in the population
        gen_best = 0.0
        for j in range(0, population_size):
            #--- MUTATION (step #3.A) ---------------------+
            # select three random vector index positions [0, popsize), not including current vector (j)
            canidates = list(range(0,population_size))
            canidates.remove(j)
            random_index = random.sample(canidates, 3)

            x_1 = population[random_index[0]][1:-1]              
            x_2 = population[random_index[1]][1:-1]              
            x_3 = population[random_index[2]][1:-1]              
            x_t = population[j]     # target individual    
            # subtract x3 from x2, and create a new vector (x_diff)
            # x_diff = [x_2_i - x_3_i for x_2_i, x_3_i in zip(x_2, x_3)]
            # x_diff = []
            # for element in range(len(x_2)):
            #     x_diff.append(x_2[element] - x_3[element])
            x_diff = x_2 - x_3
            # multiply x_diff by the mutation factor (F) and add to x_1
            # v_donor = [x_1_i + (mutate * x_diff_i) for x_1_i, x_diff_i in zip(x_1, x_diff)]
            # Cast elements of v_donor to integers before appending to v_trial
            # v_donor = np.array([int(round(x_1_i + (mutate * x_diff_i))) for x_1_i, x_diff_i in zip(x_1, x_diff)])
            v_donor = x_1 + (mutate * x_diff)
            v_donor = v_donor.astype(np.int32)
            # v_donor = []
            # for element in range(len(x_1)):
            #    v_donor.append(int(round(x_1[element] + (mutate * x_diff[element])))) 
            # v_donor = np.array(v_donor)
            #ensure_bounds and duplacetes
            for i in range(len(v_donor)):
                # Check if the value appears again in the list beyond the current index
                if np.sum(v_donor == v_donor[i]) >= 2 or v_donor[i] not in range(1, num_customers + 1):
                    for j in range(1, num_customers + 1):
                        if j not in v_donor:
                            v_donor[i] = j  # Replace duplicate with missing number
                            break
            v_donor = np.concatenate(([0], v_donor, [0]))            
            #--- RECOMBINATION (step #3.B) ----------------+
            v_trial = []
            for k in range(len(x_t)):
                if random.random() <= recombination:
                    v_trial.append(v_donor[k])
                else:
                    v_trial.append(x_t[k])
            v_trial = np.array(v_trial)
            
            #--- GREEDY SELECTION (step #3.C) -------------+
            score_trial  = calculate_distance(v_trial, coords)
            score_target = calculate_distance(x_t, coords)
            
            if score_trial < score_target:
                population[j] = v_trial
                gen_scores.append(score_trial)
            else:
                gen_scores.append(score_target)
        #--- SCORE KEEPING --------------------------------+
        gen_avg = sum(gen_scores) / population_size                         # current generation avg. fitness
        gen_best = min(gen_scores)                                  # fitness of best individual
        gen_sol = population[gen_scores.index(min(gen_scores))]     # solution of best individual
        if gen % 50 == 0 or gen == generations:
            print(f"Generation {gen + 1}/{generations}: Best Distance = {gen_best:.2f}")
            best_distance_arr.append(gen_best)
    best_distance_arr = np.array(best_distance_arr)
    print("maybe a best route:", gen_sol)
    print("a maybe  best distance  ??:", gen_best)
    return gen_sol, best_distance_arr

In [94]:
best_route_de, best_distance_arr_de = run_de(coords)

Generation 1/1000: Best Distance = 755.60
Generation 51/1000: Best Distance = 768.69
Generation 101/1000: Best Distance = 825.45
Generation 151/1000: Best Distance = 756.27
Generation 201/1000: Best Distance = 803.91
Generation 251/1000: Best Distance = 758.21
Generation 301/1000: Best Distance = 757.66
Generation 351/1000: Best Distance = 801.33
Generation 401/1000: Best Distance = 812.92
Generation 451/1000: Best Distance = 803.98
Generation 501/1000: Best Distance = 805.16
Generation 551/1000: Best Distance = 846.20
Generation 601/1000: Best Distance = 712.47
Generation 651/1000: Best Distance = 777.43
Generation 701/1000: Best Distance = 697.74
Generation 751/1000: Best Distance = 775.03
Generation 801/1000: Best Distance = 771.75
Generation 851/1000: Best Distance = 790.27
Generation 901/1000: Best Distance = 800.63
Generation 951/1000: Best Distance = 794.41
maybe a best route: [ 0 13 20  4  2  5 18 11 17  7  1  8 12 10 19 15 14  3 16  6  9  0]
a maybe  best distance  ??: 725.529

In [95]:
plot_route(coords_df, best_route_de)

In [96]:
import plotly.express as px

loss_df = pd.DataFrame({'Generation': np.arange(0, len(best_distance_arr_de) * 50, 50), 'Best Distance': best_distance_arr_de})

plot = px.line(loss_df, x='Generation', y='Best Distance', title='Best Distance Over Generations')
plot.show()


# **************************************************

# old mutation function for differntial 

In [97]:

# def mutate(diff_population, target_idx, F):

#     pop_size = len(diff_population)

#     indices = list(range(pop_size))
#     indices.remove(target_idx) # so i dont chooce target for calculation
#     x1, x2, x3 = diff_population[np.random.choice(indices, 3, replace=False)]

#     mutant = x3 + F * (x2 - x1)


#     mutant = np.round(mutant).astype(int)# formula can result in float values so i round them to int

#     return mutant

# F = 0.8  
# target_idx = 0  
# mutant_vector = mutate(diff_population, target_idx, F)
# print(mutant_vector)


In [98]:
# mutant_vector = validate_and_repair(mutant_vector, num_customers)

In [99]:
# CR = 0.9  
# target = diff_population[0]  
# mutant = mutate(diff_population, 0, F=0.8)  

# if np.random.rand() < CR:
#     child = crossover(target, mutant)  
# else:
#     child = target  
# child 